In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

!pip install backtrader
import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from google.colab import drive
drive.mount('/content/gdrive')

# Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    # RSI parameters    
    params = (
        ('ema_long_period', 26),
        ('ema_short_period',12),
        ('printlog', False),
    )
    

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
       
        
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.ema_long = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_long_period) #kevin technical indicator chosen 
        self.ema_short = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_short_period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Buy if 14 day rsi less than 20    
            if self.ema_long[0] - self.ema_short[0] > 0: # kevin to change conditions
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                    

        else:
            # Sell if 7 day rsi over 80
            if self.ema_long[0] - self.ema_short[0] < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(EMA long Period %2d, EMA short Period %2d) Ending Value %.2f' %
                 (self.params.ema_long_period, self.params.ema_short_period,  self.broker.getvalue()), doprint=True)

# Analyzers

In [ ]:
class RSIAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.ema_long = self.strategy.params.ema_long_period
        self.ema_short = self.strategy.params.ema_short_period
        self.equity = []
        
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0
        self.first_trade_open = None

    def next(self):
        pass

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()

        if trade.isclosed:
            self.num_trades += 1
            self.equity.append(self.strategy.broker.getvalue())
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        equity = np.asarray([self.init_cash,] + self.equity)
        
        results = {
            'params': (self.ema_long, self.ema_short),
            'profit': self.final_val - self.init_cash,
            'num_trades': self.num_trades,
            'trades': np.diff(equity).tolist()
        }
        
        return results

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.rsiA.get_analysis()
        params.append(r['params'])
        n_trades.append(r['num_trades'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join(r'/content/gdrive/My Drive/DecisiveWorkflowResearch/data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

# In-sample Parameters

In [ ]:
def get_periods(start_is, is_duration, oos_duration, wf_offset,wf_cycles):
    periods=[]
    start_is=pd.Timestamp(start_is)
    start_oos=start_is+pd.DateOffset(**is_duration)
    end_oos=start_oos+pd.DateOffset(**oos_duration)
    for i in range(wf_cycles):
        periods.append({'run':i,
                        'is':(start_is, start_oos),
                        'oos':(start_oos, end_oos)})
        start_is+=pd.DateOffset(**wf_offset)
        start_oos+=pd.DateOffset(**wf_offset)
        end_oos+=pd.DateOffset(**wf_offset)
    return periods
    
periods= get_periods(
    start_is = "2016-01-01", 
    is_duration={'months':3}, 
    oos_duration={'months':1},
    wf_offset={'months':3},
    wf_cycles=13
)

periods

In [ ]:


best_oos_params = []
for oos_is in periods:
    start_date, end_date = oos_is['is']
    run_num = oos_is['run']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
        TestStrategy,
        ema_long_period=range(20, 40),
        ema_short_period = range(3,20),)
    
    cerebro.addanalyzer(RSIAnalyzer, _name='rsiA')

    cerebro.optreturn = False
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    best_param, best_profit, best_trades, best_equity = best_result_from_cerebro_opti_run(result)

    best_oos_params.append({'train_param': best_param, 
                                      'train_profit': best_profit, 
                                      'train_numtrades': best_trades, 
                                      'train_tradeslist': best_equity, 
                                      'train_period': oos_is['is'],
                                      'test_period': oos_is['oos'],
                                     })
    
    print('')

print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

In [ ]:
trades_list = np.asarray(best_oos_params[4]['train_tradeslist'])
cumsum = trades_list.cumsum()
plt.plot(cumsum)

# Out-of-sample Parameters

In [ ]:
run_num = 0
for best in best_oos_params:
    start_date, end_date = best['test_period']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy, ema_long_period=best['train_param'][0], 
                                    ema_short_period = best['train_param'][1],
                                   )

    
    cerebro.addanalyzer(RSIAnalyzer, _name='rsiA')

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    r = result[0].analyzers.rsiA.get_analysis()
    best_oos_params[run_num]['test_numtrades'] = r['num_trades']
    best_oos_params[run_num]['test_tradeslist'] = r['trades']
    best_oos_params[run_num]['test_profit'] = r['profit']
    run_num += 1

In [ ]:
print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

# Walkforward

In [ ]:
def days_from_date_tuple(mytuple):
    #insample_days_start = datetime.datetime.strptime(mytuple[0], '%Y-%m-%d')
    #insample_days_end = datetime.datetime.strptime(mytuple[1], '%Y-%m-%d')
    insample_days_start = mytuple[0]
    insample_days_end = mytuple[1]
    insample_days = (insample_days_end - insample_days_start).days
    return insample_days

oos_trades = []
wfe_list = []
for values in best_oos_params:
    oos_trades_list = values['test_tradeslist']
    oos_trades.extend(oos_trades_list)
    
    insample_days = days_from_date_tuple(values['train_period'])
    oos_days = days_from_date_tuple(values['test_period'])

    insample_annual_profit = 365/insample_days * values['train_profit']
    oos_annual_profit = 365/oos_days * values['test_profit']

    walkforward_efficiency_pct = round(oos_annual_profit * 100 / insample_annual_profit, 1)
    
    wfe_list.append(walkforward_efficiency_pct)

    print('Run {} WFE:{}'.format(run_num, walkforward_efficiency_pct))
    print('\t IS_DAYS: {} OOS_DAYS: {} IS_PROFIT: {:0.2f} OOS_PROFIT: {:0.2f}'.format(insample_days, oos_days,
                                                                            insample_annual_profit, oos_annual_profit))

print('Average WFE: {}%'.format(np.asarray(wfe_list).mean()))
starting_equity = 20000
bad_wf_method = np.asarray(oos_trades).cumsum()
plt.plot(bad_wf_method)
plt.title('Incorrect Implementation of Walkforward Equity Curve')

In [ ]:
# Create a Stratey
class WalkforwardStrategy(bt.Strategy):

    params = (
        ('ema_long_period', 24),
        ('ema_short_period',8),
        ('printlog', False),
        ('live', False),
        ('walkforward', None),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add Exponential MovingAverageSimple indicator
        self.ema_long = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_long_period) #kevin technical indicator chosen 
        self.ema_short = bt.indicators.MovingAverageExponential(self.datas[0], period=self.params.ema_short_period)

        
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
        
        
        self.current_row = None
        self.wfsma = []
        
        if self.params.walkforward:
            for run in self.params.walkforward:
                self.wfsma.append({
                    "ema": bt.indicators.MovingAverageExponential(self.datas[0], period=run['train_param'][0]),
                    
                    'test_period': run['test_period'],
                    'train_param': run['train_param'],
                }) 

        print(pformat(self.wfsma))
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        # Walk-forward logic
        for row in self.wfsma:
            start, end = row['test_period']
            #period_start = datetime.datetime.strptime(start, '%Y-%m-%d')
            #period_end = datetime.datetime.strptime(end, '%Y-%m-%d')
            period_start = start
            period_end = end
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Buy    
            if self.ema_long[0] - self.ema_short[0] > 0:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                    

        else:
            # Sell
            if self.ema_long[0] - self.ema_short[0] < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()   
                
    def stop(self):
        self.log('(EMA long Period %2d, EMA short Period %2d) Ending Value %.2f' %
                 (self.params.ema_long_period, self.params.ema_short_period,  self.broker.getvalue()), doprint=True)


In [ ]:

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    #rsiPeriod=21, 
                    walkforward=best_oos_params,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join(r'/content/gdrive/My Drive/DecisiveWorkflowResearch/data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = best_oos_params[0]['test_period'][0]
end_date = best_oos_params[-1]['test_period'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(RSIAnalyzer, _name='rsiA')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.rsiA.get_analysis()
cumsum = np.asarray(r['trades']).cumsum()

plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')
avg_wfe = np.round(np.asarray(wfe_list).mean(),1)
print('Average WFE: {}%'.format(avg_wfe))